**Part1**: Introduction to the problem

*There are 7 workers, 7 days and two production lines with two shifts at each line*. And out of this statement you can build your binary objective function - binary, because a single shift is assigned (1) or not assigned (0). So the objective function consists of `7*7*2*2=196` elements.


And of course the workers have different qualififactions which means that they cannot work on every production line (1 is allowed ; 0 is not allowed) and additionally the workers have 
preferences on which shifts they like to work (1) and not (0).

In [15]:
# Qualifications of the workers regarding production line 1 and production line 2
qualification=matrix(c(1,1,0,1,1,1,1,0,1,0,1,0,0,1),ncol=2,byrow=TRUE)
colnames(qualification)=c("line_1","line_2")
rownames(qualification)=c("worker_1","worker_2","worker_3","worker_4","worker_5","worker_6","worker_7")
qualification

,line_1,line_2
worker_1,1,1
worker_2,0,1
worker_3,1,1
worker_4,1,0
worker_5,1,0
worker_6,1,0
worker_7,0,1


In [16]:
# Now let's have a look at the preferences of the workers
preferences=matrix(c(1,1,1,0,1,0,1,1,1,1,1,1,0,1,rep(0,times=14),		# shifts worker_1
                     1,1,0,1,1,0,1,1,1,1,1,1,0,1,rep(0,times=14),		# shifts worker_2
                     1,1,1,1,1,1,1,0,1,1,1,1,1,0,rep(0,times=14),		# shifts worker_3
                     1,0,1,1,0,1,1,1,0,1,1,1,1,1,rep(0,times=14),		# shifts worker_4
                     1,0,1,1,1,1,1,1,1,1,1,1,1,0,rep(0,times=14),		# shifts worker_5
                     1,1,1,1,1,1,0,1,1,1,0,1,1,0,rep(0,times=14),		# shifts worker_6
                     0,0,1,1,1,1,1,1,1,1,1,1,1,1),nrow=7,byrow=FALSE)	# shifts worker_7
rownames(preferences)=c("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
colnames(preferences)=c("ES_W1","LS_W1","--","--",	# ES (early shift) and LS (late shift) for worker_1
                        "ES_W2","LS_W2","--","--",	# ES (early shift) and LS (late shift) for worker_2
                        "ES_W3","LS_W3","--","--",	# ES (early shift) and LS (late shift) for worker_3
                        "ES_W4","LS_W4","--","--",	# ES (early shift) and LS (late shift) for worker_4
                        "ES_W5","LS_W5","--","--",	# ES (early shift) and LS (late shift) for worker_5
                        "ES_W6","LS_W6","--","--",	# ES (early shift) and LS (late shift) for worker_6
                        "ES_W7","LS_W7")			# ES (early shift) and LS (late shift) for worker_7
preferences

,ES_W1,LS_W1,--,--,ES_W2,LS_W2,--,--,ES_W3,LS_W3,⋯,ES_W5,LS_W5,--,--,ES_W6,LS_W6,--,--,ES_W7,LS_W7
Monday,1,1,0,0,1,1,0,0,1,0,⋯,1,1,0,0,1,1,0,0,0,1
Tuesday,1,1,0,0,1,1,0,0,1,1,⋯,0,1,0,0,1,1,0,0,0,1
Wednesday,1,1,0,0,0,1,0,0,1,1,⋯,1,1,0,0,1,1,0,0,1,1
Thursday,0,1,0,0,1,1,0,0,1,1,⋯,1,1,0,0,1,0,0,0,1,1
Friday,1,1,0,0,1,1,0,0,1,1,⋯,1,1,0,0,1,1,0,0,1,1
Saturday,0,0,0,0,0,0,0,0,1,1,⋯,1,1,0,0,1,1,0,0,1,1
Sunday,1,1,0,0,1,1,0,0,1,0,⋯,1,0,0,0,0,0,0,0,1,1


**Part 2**: Implementing the matrix mat - left hand side of the constraints

These qualifications and preferences of the workers lead to some constraints which are stored in matrix **mat**.

* shifts that are proper according to the qualifications and preferences of the workers (quantity of constraints: 7)
	* mat \* x <= 14
* the single worker should have at least 3 shifts per week (quantity of constraints: 7)
	* the same ones like above but multiplied by `-1`
	* mat \* x >= 3
* an early shift (ES) isn't allowed if before an late shift (LS) is planned for the same worker (quantity of constraints: 7\*28=196)
	* 28 constraints per worker
	* mat \* x <= 1
* only one worker per shift is allowed (quantity of constraints: 7\*4=28)
	* mat \* x <= 1
* one worker can only work in one early shift (ES) or late shift (LS) of one production line (quantity of constraints: 98)
	* mat \* x <= 1
* and just for fun I made two additional constraints that worker_1 and worker_4 have 3 days consecutive free (quantity of constraints: 2)
	* mat \* x == 0



One annotation about the structure of the objective function, which is important to understand the single rows of matrix **mat**:

* the first four entries are for *Monday-Worker_1-Line1_(early and late shift)-Line2_(early and late shift)* represented through 1\*x[1] + 1\*x[2] + 1\*x[3] + 1\*x[4]
	* 1\*x[1] is L1_ES (line 1 early shift)
	* 1\*x[2] is L1_LS (line 1 late shift)
	* 1\*x[3] is L2_ES (line 2 early shift)
	* 1\*x[4] is L2_LS (line 2 late shift)
* the second four entries are for *Monday-Worker_2-Line1_(early and late shift)-Line2_(early and late shift)* represented through 1\*x[5] + 1\*x[6] + 1\*x[7] + 1\*x[8]
* the third four entries are for *Monday-Worker_3-Line1_(early and late shift)-Line2_(early and late shift)* represented through 1\*x[9] + 1\*x[10] + 1\*x[11] + 1\*x[12]
* ...
* *Tuesday-Worker_1-Line1_(early and late shift)-Line2_(early and late shift)* represented through 1\*x[29] + 1\*x[30] + 1\*x[31] + 1\*x[32]
* ...
* *Wednesday-Worker_1-Line1_(early and late shift)-Line2_(early and late shift)* represented through 1\*x[57] + 1\*x[58] + 1\*x[59] + 1\*x[60]
* ...
* ...
* *Sunday-Worker_7-Line1_(early and late shift)-Line2_(early and late shift)* represented through 1\*x[193] + 1\*x[194] + 1\*x[195] + 1\*x[196]

In [17]:
# And now let's make the matrix mat
mat = matrix(c( # shifts that are proper according to the qualifications and preferences of the workers
		rep(0,times=0*4), 1, 1, 1, 1, rep(0,times=6*4), rep(0,times=0*4), 1, 1, 1, 1, rep(0,times=6*4), rep(0,times=0*4), 1, 1, 1, 1, rep(0,times=6*4), rep(0,times=0*4), 0, 1, 0, 1, rep(0,times=6*4), rep(0,times=0*4), 1, 1, 1, 1, rep(0,times=6*4), rep(0,times=0*4), 0, 0, 0, 0, rep(0,times=6*4), rep(0,times=0*4), 1, 1, 1, 1, rep(0,times=6*4),
        rep(0,times=1*4), 0, 0, 1, 1, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 1, 1, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 0, 1, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 1, 1, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 1, 1, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 0, 0, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 1, 1, rep(0,times=5*4),
        rep(0,times=2*4), 1, 0, 1, 0, rep(0,times=4*4), rep(0,times=2*4), 1, 1, 1, 1, rep(0,times=4*4), rep(0,times=2*4), 1, 1, 1, 1, rep(0,times=4*4), rep(0,times=2*4), 1, 1, 1, 1, rep(0,times=4*4), rep(0,times=2*4), 1, 1, 1, 1, rep(0,times=4*4), rep(0,times=2*4), 1, 1, 1, 1, rep(0,times=4*4), rep(0,times=2*4), 1, 0, 1, 0, rep(0,times=4*4),
	    rep(0,times=3*4), 1, 1, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 0, 0, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 1, 1, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 1, 1, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 0, 1, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 1, 1, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 1, 1, 0, 0, rep(0,times=3*4),		
        rep(0,times=4*4), 1, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 0, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 1, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 1, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 1, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 1, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 1, 0, 0, 0, rep(0,times=2*4),	
        rep(0,times=5*4), 1, 1, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 1, 1, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 1, 1, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 1, 0, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 1, 1, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 1, 1, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 0, 0, 0, 0, rep(0,times=1*4),
        rep(0,times=6*4), 0, 0, 0, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 0, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 1, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 1, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 1, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 1, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 1, 1, rep(0,times=0*4),
		######
		######
		######
		######
		# the single worker should have at least 3 shifts per week
		rep(0,times=0*4), 1, 1, 1, 1, rep(0,times=6*4), rep(0,times=0*4), 1, 1, 1, 1, rep(0,times=6*4), rep(0,times=0*4), 1, 1, 1, 1, rep(0,times=6*4), rep(0,times=0*4), 0, 1, 0, 1, rep(0,times=6*4), rep(0,times=0*4), 1, 1, 1, 1, rep(0,times=6*4), rep(0,times=0*4), 0, 0, 0, 0, rep(0,times=6*4), rep(0,times=0*4), 1, 1, 1, 1, rep(0,times=6*4),
        rep(0,times=1*4), 0, 0, 1, 1, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 1, 1, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 0, 1, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 1, 1, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 1, 1, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 0, 0, rep(0,times=5*4), rep(0,times=1*4), 0, 0, 1, 1, rep(0,times=5*4),
        rep(0,times=2*4), 1, 0, 1, 0, rep(0,times=4*4), rep(0,times=2*4), 1, 1, 1, 1, rep(0,times=4*4), rep(0,times=2*4), 1, 1, 1, 1, rep(0,times=4*4), rep(0,times=2*4), 1, 1, 1, 1, rep(0,times=4*4), rep(0,times=2*4), 1, 1, 1, 1, rep(0,times=4*4), rep(0,times=2*4), 1, 1, 1, 1, rep(0,times=4*4), rep(0,times=2*4), 1, 0, 1, 0, rep(0,times=4*4),
	    rep(0,times=3*4), 1, 1, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 0, 0, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 1, 1, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 1, 1, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 0, 1, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 1, 1, 0, 0, rep(0,times=3*4), rep(0,times=3*4), 1, 1, 0, 0, rep(0,times=3*4),
        rep(0,times=4*4), 1, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 0, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 1, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 1, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 1, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 1, 1, 0, 0, rep(0,times=2*4), rep(0,times=4*4), 1, 0, 0, 0, rep(0,times=2*4),
        rep(0,times=5*4), 1, 1, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 1, 1, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 1, 1, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 1, 0, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 1, 1, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 1, 1, 0, 0, rep(0,times=1*4), rep(0,times=5*4), 0, 0, 0, 0, rep(0,times=1*4),
        rep(0,times=6*4), 0, 0, 0, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 0, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 1, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 1, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 1, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 1, 1, rep(0,times=0*4), rep(0,times=6*4), 0, 0, 1, 1, rep(0,times=0*4),
		######
		######
		######
		######
		# an early shift (ES) isn't allowed if before an late shift (LS) is planned for the same worker
		#   worker_1 (Monday to Tuesday)
	    rep(0,times=(0*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 1, 0, rep(0,times=(6*4)), rep(0,times=(5*(7*4))),
        rep(0,times=(0*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 1, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(5*(7*4))),
	    rep(0,times=(0*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 1, 0, rep(0,times=(6*4)), rep(0,times=(5*(7*4))),
        rep(0,times=(0*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)), rep(0,times=(0*4)), 1, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(5*(7*4))),
		#   worker_1 (Tues to Wed)
	    rep(0,times=(1*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 1, 0, rep(0,times=(6*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 1, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 1, 0, rep(0,times=(6*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)), rep(0,times=(0*4)), 1, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(4*(7*4))),
		#   worker_1 (Wed to Thu)
		rep(0,times=(2*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(3*(7*4))),
	 	rep(0,times=(2*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(3*(7*4))),
		#   worker_1 (Thu to Fri)
		rep(0,times=(3*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 1, 0, rep(0,times=(6*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 1, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 1, 0, rep(0,times=(6*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)), rep(0,times=(0*4)), 1, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(2*(7*4))),
		#   worker_1 (Fri to Sat)
		rep(0,times=(4*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(1*(7*4))),
		rep(0,times=(4*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(1*(7*4))),
	 	rep(0,times=(4*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(1*(7*4))),
		#   worker_1 (Sat to Sun)
		rep(0,times=(5*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 1, 0, rep(0,times=(6*4)), rep(0,times=(0*(7*4))),
		rep(0,times=(5*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 1, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 0, 0, 1, 0, rep(0,times=(6*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*4)), 1, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(0*(7*4))),
		#   worker_1 (Sun to Mon)
 		rep(0,times=(0*4)), 0, 0, 1, 0, rep(0,times=(6*4)), rep(0,times=(5*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)),
 		rep(0,times=(0*4)), 1, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(5*(7*4))), rep(0,times=(0*4)), 0, 1, 0, 0, rep(0,times=(6*4)),
 		rep(0,times=(0*4)), 0, 0, 1, 0, rep(0,times=(6*4)), rep(0,times=(5*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)),
		rep(0,times=(0*4)), 1, 0, 0, 0, rep(0,times=(6*4)), rep(0,times=(5*(7*4))), rep(0,times=(0*4)), 0, 0, 0, 1, rep(0,times=(6*4)),
		######
		######
		#   worker_2 (Mon to Tues)
		rep(0,times=(0*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 1, 0, rep(0,times=(5*4)), rep(0,times=(5*(7*4))),
		rep(0,times=(0*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(5*(7*4))),
 		rep(0,times=(0*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 1, 0, rep(0,times=(5*4)), rep(0,times=(5*(7*4))),
 		rep(0,times=(0*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(5*(7*4))),
		#   worker_2 (Tues to Wed)
 		rep(0,times=(1*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(4*(7*4))),
		rep(0,times=(1*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(4*(7*4))),
		#   worker_2 (Wed to Thu)
 		rep(0,times=(2*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 1, 0, rep(0,times=(5*4)), rep(0,times=(3*(7*4))),
		rep(0,times=(2*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 1, 0, rep(0,times=(5*4)), rep(0,times=(3*(7*4))),
		rep(0,times=(2*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(3*(7*4))),
		#   worker_2 (Thu to Fri)
		rep(0,times=(3*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 1, 0, rep(0,times=(5*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 1, 0, rep(0,times=(5*4)), rep(0,times=(2*(7*4))),
		rep(0,times=(3*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(2*(7*4))),
		#   worker_2 (Fri to Sat)
	 	rep(0,times=(4*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*(7*4))),
		rep(0,times=(4*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*(7*4))),
		#   worker_2 (Sat to Sun)
	 	rep(0,times=(5*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 1, 0, rep(0,times=(5*4)), rep(0,times=(0*(7*4))),
		rep(0,times=(5*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 1, 0, rep(0,times=(5*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(0*(7*4))),
		#   worker_2 (Sun to Mon)
		rep(0,times=(1*4)), 0, 0, 1, 0, rep(0,times=(5*4)), rep(0,times=(5*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)),
 		rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(5*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)),
		rep(0,times=(1*4)), 0, 0, 1, 0, rep(0,times=(5*4)), rep(0,times=(5*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)),
 		rep(0,times=(1*4)), 0, 0, 0, 0, rep(0,times=(5*4)), rep(0,times=(5*(7*4))), rep(0,times=(1*4)), 0, 0, 0, 1, rep(0,times=(5*4)),
		######
		######
		#   worker_3 (Mon to Tues)
		rep(0,times=(0*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(5*(7*4))),
		rep(0,times=(0*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(5*(7*4))),
	 	rep(0,times=(0*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(5*(7*4))),
	 	rep(0,times=(0*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(5*(7*4))),
		#   worker_3 (Tues to Wed)
 		rep(0,times=(1*(7*4))), rep(0,times=(2*4)), 0, 1, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(2*4)), 0, 1, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(4*(7*4))),
		rep(0,times=(1*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(4*(7*4))),
		#   worker_3 (Wed to Thu)
 		rep(0,times=(2*(7*4))), rep(0,times=(2*4)), 0, 1, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(3*(7*4))),
		rep(0,times=(2*(7*4))), rep(0,times=(2*4)), 0, 1, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(3*(7*4))),
		#   worker_3 (Thu to Fri)
 		rep(0,times=(3*(7*4))), rep(0,times=(2*4)), 0, 1, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(2*4)), 0, 1, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(2*(7*4))),
		rep(0,times=(3*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*(7*4))),
		#   worker_3 (Fri to Sat)
		rep(0,times=(4*(7*4))), rep(0,times=(2*4)), 0, 1, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(2*4)), 0, 1, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(1*(7*4))),
		#   worker_3 (Sat to Sun)
 		rep(0,times=(5*(7*4))), rep(0,times=(2*4)), 0, 1, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(2*4)), 0, 1, 0, 0, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(0*(7*4))),
		#   worker_3 (Sun to Mon)
 		rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(5*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 0, rep(0,times=(4*4)),
 		rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(5*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 0, rep(0,times=(4*4)),
 		rep(0,times=(2*4)), 0, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(5*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 0, rep(0,times=(4*4)),
 		rep(0,times=(2*4)), 1, 0, 0, 0, rep(0,times=(4*4)), rep(0,times=(5*(7*4))), rep(0,times=(2*4)), 0, 0, 0, 0, rep(0,times=(4*4)),
		######
		######
		#   worker_4 (Mon to Tues)
 		rep(0,times=(0*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(5*(7*4))),
 		rep(0,times=(0*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(5*(7*4))),
 		rep(0,times=(0*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(5*(7*4))),
		rep(0,times=(0*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(5*(7*4))),
		#   worker_4 (Tues to Wed)
 		rep(0,times=(1*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 1, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(4*(7*4))),
		rep(0,times=(1*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 1, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(4*(7*4))),
		#   worker_4 (Wed to Thu)
 		rep(0,times=(2*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 1, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*(7*4))),
		rep(0,times=(2*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 1, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*(7*4))),
		#   worker_4 (Thu to Fri)
		rep(0,times=(3*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(2*(7*4))),
		#   worker_4 (Fri to Sat)
	 	rep(0,times=(4*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 1, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 1, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(1*(7*4))),
		#   worker_4 (Sat to Sun)
		rep(0,times=(5*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 1, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(3*4)), 1, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(0*(7*4))),
		#   worker_4 (Sun to Mon)
 		rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(5*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)),
 		rep(0,times=(3*4)), 1, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(5*(7*4))), rep(0,times=(3*4)), 0, 1, 0, 0, rep(0,times=(3*4)),
		rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(5*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)),
 		rep(0,times=(3*4)), 1, 0, 0, 0, rep(0,times=(3*4)), rep(0,times=(5*(7*4))), rep(0,times=(3*4)), 0, 0, 0, 0, rep(0,times=(3*4)),
		######
		######
		#   worker_5 (Mon to Tues)
 		rep(0,times=(0*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(5*(7*4))),
 		rep(0,times=(0*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(5*(7*4))),
 		rep(0,times=(0*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(5*(7*4))),
 		rep(0,times=(0*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(5*(7*4))),
		#   worker_5 (Tues to Wed)
 		rep(0,times=(1*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*(7*4))),
		#   worker_5 (Wed to Thu)
 		rep(0,times=(2*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(3*(7*4))),
		#   worker_5 (Thu to Fri)
 		rep(0,times=(3*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(2*(7*4))),
		#   worker_5 (Fri to Sat)
 		rep(0,times=(4*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(1*(7*4))),
		#   worker_5 (Sat to Sun)
		rep(0,times=(5*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(4*4)), 0, 1, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(0*(7*4))),
		#   worker_5 (Sun to Mon)
	 	rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(5*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)),
 		rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(5*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)),
 		rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(5*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)),
 		rep(0,times=(4*4)), 1, 0, 0, 0, rep(0,times=(2*4)), rep(0,times=(5*(7*4))), rep(0,times=(4*4)), 0, 0, 0, 0, rep(0,times=(2*4)),
		######
		######
		#   worker_6 (Mon to Tues)
		rep(0,times=(0*(7*4))), rep(0,times=(5*4)), 0, 1, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*(7*4))),
		rep(0,times=(0*(7*4))), rep(0,times=(5*4)), 0, 1, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*(7*4))),
		rep(0,times=(0*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*(7*4))),
 		rep(0,times=(0*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*(7*4))),
		#   worker_6 (Tues to Wed)
	 	rep(0,times=(1*(7*4))), rep(0,times=(5*4)), 0, 1, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(4*(7*4))),
	 	rep(0,times=(1*(7*4))), rep(0,times=(5*4)), 0, 1, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(4*(7*4))),
		#   worker_6 (Wed to Thu)
		rep(0,times=(2*(7*4))), rep(0,times=(5*4)), 0, 1, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(3*(7*4))),
		rep(0,times=(2*(7*4))), rep(0,times=(5*4)), 0, 1, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(3*(7*4))),
		#   worker_6 (Thu to Fri)
 		rep(0,times=(3*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(2*(7*4))),
		rep(0,times=(3*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(2*(7*4))),
		rep(0,times=(3*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(2*(7*4))),
		#   worker_6 (Fri to Sat)
	 	rep(0,times=(4*(7*4))), rep(0,times=(5*4)), 0, 1, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(5*4)), 0, 1, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(1*(7*4))),
		#   worker_6 (Sat to Sun)
 		rep(0,times=(5*(7*4))), rep(0,times=(5*4)), 0, 1, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(5*4)), 0, 1, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(0*(7*4))),
		#   worker_6 (Sun to Mon)
		rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)),
	 	rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)),
 		rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)),
 		rep(0,times=(5*4)), 1, 0, 0, 0, rep(0,times=(1*4)), rep(0,times=(5*(7*4))), rep(0,times=(5*4)), 0, 0, 0, 0, rep(0,times=(1*4)),
		######
		######
		#   worker_7 (Mon to Tues)
	 	rep(0,times=(0*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(5*(7*4))),
		rep(0,times=(0*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(5*(7*4))),
		rep(0,times=(0*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(5*(7*4))),
		rep(0,times=(0*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(5*(7*4))),
		#   worker_7 (Tues to Wed)
 		rep(0,times=(1*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 1, 0, rep(0,times=(0*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 1, 0, rep(0,times=(0*4)), rep(0,times=(4*(7*4))),
 		rep(0,times=(1*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(4*(7*4))),
		#   worker_7 (Wed to Thu)
 		rep(0,times=(2*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 1, 0, rep(0,times=(0*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 1, 0, rep(0,times=(0*4)), rep(0,times=(3*(7*4))),
 		rep(0,times=(2*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(3*(7*4))),
		#   worker_7 (Thu to Fri)
		rep(0,times=(3*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 1, 0, rep(0,times=(0*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 1, 0, rep(0,times=(0*4)), rep(0,times=(2*(7*4))),
 		rep(0,times=(3*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(2*(7*4))),
		#   worker_7 (Fri to Sat)
	 	rep(0,times=(4*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 1, 0, rep(0,times=(0*4)), rep(0,times=(1*(7*4))),
		rep(0,times=(4*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 1, 0, rep(0,times=(0*4)), rep(0,times=(1*(7*4))),
 		rep(0,times=(4*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(1*(7*4))),
		#   worker_7 (Sat to Sun)
 		rep(0,times=(5*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 1, 0, rep(0,times=(0*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(0*(7*4))),
 		rep(0,times=(5*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 1, 0, rep(0,times=(0*4)), rep(0,times=(0*(7*4))),
		rep(0,times=(5*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(0*(7*4))),
		#   worker_7 (Sun to Mon)
 		rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(5*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)),
		rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(5*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)),
 		rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(5*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)),
 		rep(0,times=(6*4)), 0, 0, 0, 0, rep(0,times=(0*4)), rep(0,times=(5*(7*4))), rep(0,times=(6*4)), 0, 0, 0, 1, rep(0,times=(0*4)),
		######
		######
		######
		######
		# only one worker per shift is allowed
		# Monday
 		rep(0,times=(0*(4*7))), rep(c(1,0,0,0),times=7), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(c(0,1,0,0),times=7), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(c(0,0,1,0),times=7), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(c(0,0,0,1),times=7), rep(0,times=(6*(4*7))),
		# Tuesday
 		rep(0,times=(1*(4*7))), rep(c(1,0,0,0),times=7), rep(0,times=(5*(4*7))),
 		rep(0,times=(1*(4*7))), rep(c(0,1,0,0),times=7), rep(0,times=(5*(4*7))),
 		rep(0,times=(1*(4*7))), rep(c(0,0,1,0),times=7), rep(0,times=(5*(4*7))),
 		rep(0,times=(1*(4*7))), rep(c(0,0,0,1),times=7), rep(0,times=(5*(4*7))),
		# Wednesday
	 	rep(0,times=(2*(4*7))), rep(c(1,0,0,0),times=7), rep(0,times=(4*(4*7))),
 		rep(0,times=(2*(4*7))), rep(c(0,1,0,0),times=7), rep(0,times=(4*(4*7))),
		rep(0,times=(2*(4*7))), rep(c(0,0,1,0),times=7), rep(0,times=(4*(4*7))),
 		rep(0,times=(2*(4*7))), rep(c(0,0,0,1),times=7), rep(0,times=(4*(4*7))),
		# Thursday
	 	rep(0,times=(3*(4*7))), rep(c(1,0,0,0),times=7), rep(0,times=(3*(4*7))),
		rep(0,times=(3*(4*7))), rep(c(0,1,0,0),times=7), rep(0,times=(3*(4*7))),
 		rep(0,times=(3*(4*7))), rep(c(0,0,1,0),times=7), rep(0,times=(3*(4*7))),
 		rep(0,times=(3*(4*7))), rep(c(0,0,0,1),times=7), rep(0,times=(3*(4*7))),
		# Friday
		rep(0,times=(4*(4*7))), rep(c(1,0,0,0),times=7), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(c(0,1,0,0),times=7), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(c(0,0,1,0),times=7), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(c(0,0,0,1),times=7), rep(0,times=(2*(4*7))),
		# Saturday
 		rep(0,times=(5*(4*7))), rep(c(1,0,0,0),times=7), rep(0,times=(1*(4*7))),
 		rep(0,times=(5*(4*7))), rep(c(0,1,0,0),times=7), rep(0,times=(1*(4*7))),
 		rep(0,times=(5*(4*7))), rep(c(0,0,1,0),times=7), rep(0,times=(1*(4*7))),
 		rep(0,times=(5*(4*7))), rep(c(0,0,0,1),times=7), rep(0,times=(1*(4*7))),
		# Sunday
 		rep(0,times=(6*(4*7))), rep(c(1,0,0,0),times=7), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(c(0,1,0,0),times=7), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(c(0,0,1,0),times=7), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(c(0,0,0,1),times=7), rep(0,times=(0*(4*7))),
		######
		######
		######
		######
		# one worker can only work in one early shift (ES) or late shift (LS) of one production line
		# Monday early shift (ES) (for all workers)
 		rep(0,times=(0*(4*7))), rep(0,times=(4*0)), 1, 0, 1, 0, rep(0,times=(4*6)), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(0,times=(4*1)), 1, 0, 1, 0, rep(0,times=(4*5)), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(0,times=(4*2)), 1, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(0,times=(4*3)), 1, 0, 1, 0, rep(0,times=(4*3)), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(0,times=(4*4)), 1, 0, 1, 0, rep(0,times=(4*2)), rep(0,times=(6*(4*7))),
		rep(0,times=(0*(4*7))), rep(0,times=(4*5)), 1, 0, 1, 0, rep(0,times=(4*1)), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(0,times=(4*6)), 1, 0, 1, 0, rep(0,times=(4*0)), rep(0,times=(6*(4*7))),
		# Monday late shift (LS) (for all workers)
		rep(0,times=(0*(4*7))), rep(0,times=(4*0)), 0, 1, 0, 1, rep(0,times=(4*6)), rep(0,times=(6*(4*7))),
		rep(0,times=(0*(4*7))), rep(0,times=(4*1)), 0, 1, 0, 1, rep(0,times=(4*5)), rep(0,times=(6*(4*7))),
		rep(0,times=(0*(4*7))), rep(0,times=(4*2)), 0, 1, 0, 1, rep(0,times=(4*4)), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(0,times=(4*3)), 0, 1, 0, 1, rep(0,times=(4*3)), rep(0,times=(6*(4*7))),
		rep(0,times=(0*(4*7))), rep(0,times=(4*4)), 0, 1, 0, 1, rep(0,times=(4*2)), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(0,times=(4*5)), 0, 1, 0, 1, rep(0,times=(4*1)), rep(0,times=(6*(4*7))),
 		rep(0,times=(0*(4*7))), rep(0,times=(4*6)), 0, 1, 0, 1, rep(0,times=(4*0)), rep(0,times=(6*(4*7))),
		# Tuesday ES (for all workers)
 		rep(0,times=(1*(4*7))), rep(0,times=(4*0)), 1, 0, 1, 0, rep(0,times=(4*6)), rep(0,times=(5*(4*7))),
		rep(0,times=(1*(4*7))), rep(0,times=(4*1)), 1, 0, 1, 0, rep(0,times=(4*5)), rep(0,times=(5*(4*7))),
		rep(0,times=(1*(4*7))), rep(0,times=(4*2)), 1, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(5*(4*7))),
		rep(0,times=(1*(4*7))), rep(0,times=(4*3)), 1, 0, 1, 0, rep(0,times=(4*3)), rep(0,times=(5*(4*7))),
		rep(0,times=(1*(4*7))), rep(0,times=(4*4)), 1, 0, 1, 0, rep(0,times=(4*2)), rep(0,times=(5*(4*7))),
 		rep(0,times=(1*(4*7))), rep(0,times=(4*5)), 1, 0, 1, 0, rep(0,times=(4*1)), rep(0,times=(5*(4*7))),
 		rep(0,times=(1*(4*7))), rep(0,times=(4*6)), 1, 0, 1, 0, rep(0,times=(4*0)), rep(0,times=(5*(4*7))),
		# Tuesday LS (for all workers)
	 	rep(0,times=(1*(4*7))), rep(0,times=(4*0)), 0, 1, 0, 1, rep(0,times=(4*6)), rep(0,times=(5*(4*7))),
		rep(0,times=(1*(4*7))), rep(0,times=(4*1)), 0, 1, 0, 1, rep(0,times=(4*5)), rep(0,times=(5*(4*7))),
		rep(0,times=(1*(4*7))), rep(0,times=(4*2)), 0, 1, 0, 1, rep(0,times=(4*4)), rep(0,times=(5*(4*7))),
		rep(0,times=(1*(4*7))), rep(0,times=(4*3)), 0, 1, 0, 1, rep(0,times=(4*3)), rep(0,times=(5*(4*7))),
		rep(0,times=(1*(4*7))), rep(0,times=(4*4)), 0, 1, 0, 1, rep(0,times=(4*2)), rep(0,times=(5*(4*7))),
 		rep(0,times=(1*(4*7))), rep(0,times=(4*5)), 0, 1, 0, 1, rep(0,times=(4*1)), rep(0,times=(5*(4*7))),
 		rep(0,times=(1*(4*7))), rep(0,times=(4*6)), 0, 1, 0, 1, rep(0,times=(4*0)), rep(0,times=(5*(4*7))),
		# Wednesday ES (for all workers)
		rep(0,times=(2*(4*7))), rep(0,times=(4*0)), 1, 0, 1, 0, rep(0,times=(4*6)), rep(0,times=(4*(4*7))),
 		rep(0,times=(2*(4*7))), rep(0,times=(4*1)), 1, 0, 1, 0, rep(0,times=(4*5)), rep(0,times=(4*(4*7))),
		rep(0,times=(2*(4*7))), rep(0,times=(4*2)), 1, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(4*(4*7))),
		rep(0,times=(2*(4*7))), rep(0,times=(4*3)), 1, 0, 1, 0, rep(0,times=(4*3)), rep(0,times=(4*(4*7))),
 		rep(0,times=(2*(4*7))), rep(0,times=(4*4)), 1, 0, 1, 0, rep(0,times=(4*2)), rep(0,times=(4*(4*7))),
		rep(0,times=(2*(4*7))), rep(0,times=(4*5)), 1, 0, 1, 0, rep(0,times=(4*1)), rep(0,times=(4*(4*7))),
		rep(0,times=(2*(4*7))), rep(0,times=(4*6)), 1, 0, 1, 0, rep(0,times=(4*0)), rep(0,times=(4*(4*7))),
		# Wednesday LS (for all workers)
	 	rep(0,times=(2*(4*7))), rep(0,times=(4*0)), 0, 1, 0, 1, rep(0,times=(4*6)), rep(0,times=(4*(4*7))),
		rep(0,times=(2*(4*7))), rep(0,times=(4*1)), 0, 1, 0, 1, rep(0,times=(4*5)), rep(0,times=(4*(4*7))),
		rep(0,times=(2*(4*7))), rep(0,times=(4*2)), 0, 1, 0, 1, rep(0,times=(4*4)), rep(0,times=(4*(4*7))),
 		rep(0,times=(2*(4*7))), rep(0,times=(4*3)), 0, 1, 0, 1, rep(0,times=(4*3)), rep(0,times=(4*(4*7))),
	 	rep(0,times=(2*(4*7))), rep(0,times=(4*4)), 0, 1, 0, 1, rep(0,times=(4*2)), rep(0,times=(4*(4*7))),
	 	rep(0,times=(2*(4*7))), rep(0,times=(4*5)), 0, 1, 0, 1, rep(0,times=(4*1)), rep(0,times=(4*(4*7))),
	 	rep(0,times=(2*(4*7))), rep(0,times=(4*6)), 0, 1, 0, 1, rep(0,times=(4*0)), rep(0,times=(4*(4*7))),
		# Thursday ES (for all workers)
 		rep(0,times=(3*(4*7))), rep(0,times=(4*0)), 1, 0, 1, 0, rep(0,times=(4*6)), rep(0,times=(3*(4*7))),
 		rep(0,times=(3*(4*7))), rep(0,times=(4*1)), 1, 0, 1, 0, rep(0,times=(4*5)), rep(0,times=(3*(4*7))),
 		rep(0,times=(3*(4*7))), rep(0,times=(4*2)), 1, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(3*(4*7))),
 		rep(0,times=(3*(4*7))), rep(0,times=(4*3)), 1, 0, 1, 0, rep(0,times=(4*3)), rep(0,times=(3*(4*7))),
 		rep(0,times=(3*(4*7))), rep(0,times=(4*4)), 1, 0, 1, 0, rep(0,times=(4*2)), rep(0,times=(3*(4*7))),
 		rep(0,times=(3*(4*7))), rep(0,times=(4*5)), 1, 0, 1, 0, rep(0,times=(4*1)), rep(0,times=(3*(4*7))),
		rep(0,times=(3*(4*7))), rep(0,times=(4*6)), 1, 0, 1, 0, rep(0,times=(4*0)), rep(0,times=(3*(4*7))),
		# Thursday LS (for all workers)
 		rep(0,times=(3*(4*7))), rep(0,times=(4*0)), 0, 1, 0, 1, rep(0,times=(4*6)), rep(0,times=(3*(4*7))),
		rep(0,times=(3*(4*7))), rep(0,times=(4*1)), 0, 1, 0, 1, rep(0,times=(4*5)), rep(0,times=(3*(4*7))),
		rep(0,times=(3*(4*7))), rep(0,times=(4*2)), 0, 1, 0, 1, rep(0,times=(4*4)), rep(0,times=(3*(4*7))),
		rep(0,times=(3*(4*7))), rep(0,times=(4*3)), 0, 1, 0, 1, rep(0,times=(4*3)), rep(0,times=(3*(4*7))),
 		rep(0,times=(3*(4*7))), rep(0,times=(4*4)), 0, 1, 0, 1, rep(0,times=(4*2)), rep(0,times=(3*(4*7))),
 		rep(0,times=(3*(4*7))), rep(0,times=(4*5)), 0, 1, 0, 1, rep(0,times=(4*1)), rep(0,times=(3*(4*7))),
 		rep(0,times=(3*(4*7))), rep(0,times=(4*6)), 0, 1, 0, 1, rep(0,times=(4*0)), rep(0,times=(3*(4*7))),
		# Friday ES (for all workers)
		rep(0,times=(4*(4*7))), rep(0,times=(4*0)), 1, 0, 1, 0, rep(0,times=(4*6)), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(0,times=(4*1)), 1, 0, 1, 0, rep(0,times=(4*5)), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(0,times=(4*2)), 1, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(0,times=(4*3)), 1, 0, 1, 0, rep(0,times=(4*3)), rep(0,times=(2*(4*7))),
		rep(0,times=(4*(4*7))), rep(0,times=(4*4)), 1, 0, 1, 0, rep(0,times=(4*2)), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(0,times=(4*5)), 1, 0, 1, 0, rep(0,times=(4*1)), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(0,times=(4*6)), 1, 0, 1, 0, rep(0,times=(4*0)), rep(0,times=(2*(4*7))),
		# Friday LS (for all workers)
		rep(0,times=(4*(4*7))), rep(0,times=(4*0)), 0, 1, 0, 1, rep(0,times=(4*6)), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(0,times=(4*1)), 0, 1, 0, 1, rep(0,times=(4*5)), rep(0,times=(2*(4*7))),
		rep(0,times=(4*(4*7))), rep(0,times=(4*2)), 0, 1, 0, 1, rep(0,times=(4*4)), rep(0,times=(2*(4*7))),
		rep(0,times=(4*(4*7))), rep(0,times=(4*3)), 0, 1, 0, 1, rep(0,times=(4*3)), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(0,times=(4*4)), 0, 1, 0, 1, rep(0,times=(4*2)), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(0,times=(4*5)), 0, 1, 0, 1, rep(0,times=(4*1)), rep(0,times=(2*(4*7))),
 		rep(0,times=(4*(4*7))), rep(0,times=(4*6)), 0, 1, 0, 1, rep(0,times=(4*0)), rep(0,times=(2*(4*7))),
		# Saturday ES (for all workers)
 		rep(0,times=(5*(4*7))), rep(0,times=(4*0)), 1, 0, 1, 0, rep(0,times=(4*6)), rep(0,times=(1*(4*7))),
 		rep(0,times=(5*(4*7))), rep(0,times=(4*1)), 1, 0, 1, 0, rep(0,times=(4*5)), rep(0,times=(1*(4*7))),
		rep(0,times=(5*(4*7))), rep(0,times=(4*2)), 1, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(1*(4*7))),
		rep(0,times=(5*(4*7))), rep(0,times=(4*3)), 1, 0, 1, 0, rep(0,times=(4*3)), rep(0,times=(1*(4*7))),
		rep(0,times=(5*(4*7))), rep(0,times=(4*4)), 1, 0, 1, 0, rep(0,times=(4*2)), rep(0,times=(1*(4*7))),
		rep(0,times=(5*(4*7))), rep(0,times=(4*5)), 1, 0, 1, 0, rep(0,times=(4*1)), rep(0,times=(1*(4*7))),
		rep(0,times=(5*(4*7))), rep(0,times=(4*6)), 1, 0, 1, 0, rep(0,times=(4*0)), rep(0,times=(1*(4*7))),
		# Saturday LS (for all workers)
	 	rep(0,times=(5*(4*7))), rep(0,times=(4*0)), 0, 1, 0, 1, rep(0,times=(4*6)), rep(0,times=(1*(4*7))),
		rep(0,times=(5*(4*7))), rep(0,times=(4*1)), 0, 1, 0, 1, rep(0,times=(4*5)), rep(0,times=(1*(4*7))),
	 	rep(0,times=(5*(4*7))), rep(0,times=(4*2)), 0, 1, 0, 1, rep(0,times=(4*4)), rep(0,times=(1*(4*7))),
		rep(0,times=(5*(4*7))), rep(0,times=(4*3)), 0, 1, 0, 1, rep(0,times=(4*3)), rep(0,times=(1*(4*7))),
 		rep(0,times=(5*(4*7))), rep(0,times=(4*4)), 0, 1, 0, 1, rep(0,times=(4*2)), rep(0,times=(1*(4*7))),
 		rep(0,times=(5*(4*7))), rep(0,times=(4*5)), 0, 1, 0, 1, rep(0,times=(4*1)), rep(0,times=(1*(4*7))),
 		rep(0,times=(5*(4*7))), rep(0,times=(4*6)), 0, 1, 0, 1, rep(0,times=(4*0)), rep(0,times=(1*(4*7))),
		# Sunday ES (for all workers)
 		rep(0,times=(6*(4*7))), rep(0,times=(4*0)), 1, 0, 1, 0, rep(0,times=(4*6)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*1)), 1, 0, 1, 0, rep(0,times=(4*5)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*2)), 1, 0, 1, 0, rep(0,times=(4*4)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*3)), 1, 0, 1, 0, rep(0,times=(4*3)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*4)), 1, 0, 1, 0, rep(0,times=(4*2)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*5)), 1, 0, 1, 0, rep(0,times=(4*1)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*6)), 1, 0, 1, 0, rep(0,times=(4*0)), rep(0,times=(0*(4*7))),
		# Sunday LS (for all workers)
		rep(0,times=(6*(4*7))), rep(0,times=(4*0)), 0, 1, 0, 1, rep(0,times=(4*6)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*1)), 0, 1, 0, 1, rep(0,times=(4*5)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*2)), 0, 1, 0, 1, rep(0,times=(4*4)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*3)), 0, 1, 0, 1, rep(0,times=(4*3)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*4)), 0, 1, 0, 1, rep(0,times=(4*2)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*5)), 0, 1, 0, 1, rep(0,times=(4*1)), rep(0,times=(0*(4*7))),
 		rep(0,times=(6*(4*7))), rep(0,times=(4*6)), 0, 1, 0, 1, rep(0,times=(4*0)), rep(0,times=(0*(4*7))),
		######
		######
		######
		######
		# worker_1 and worker_4 have 3 days consecutive free
		# worker_1 (Monday, Tuesday, Wednesday)
		rep(1,times=4), rep(0,times=(6*4)), rep(1,times=4), rep(0,times=(6*4)), rep(1,times=4), rep(0,times=(6*4)), rep(0,times=(4*(7*4))),
		# worker_4 (Thursday, Friday, Saturday)
 		rep(0,times=(3*(7*4))), rep(0,times=(3*4)), rep(1,times=4), rep(0,times=(3*4)), rep(0,times=(3*4)), rep(1,times=4), rep(0,times=(3*4)), rep(0,times=(3*4)), rep(1,times=4), rep(0,times=(3*4)), rep(0,times=(1*(7*4)))

              ),ncol=196,byrow=TRUE)

**Part 3**: Doing the optimization

In [18]:
# load the needed packages
#install.packages("Rglpk")
library(Rglpk)

# first objective function (only constraints and no weighting)
obj_1 = rep(1, times = (196))

The weighting of the objective function is achieved through the first 7 rows of matrix mat. Therefore they are omitted later in `res_2`.

In [19]:
# getting the preferences of the workers in a vector (qualifications and preferences are included)
preference.v=c(
#worker_1---worker_2----worker_3----worker_4----worker_5----worker_6----worker_7--
1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,	# Monday
1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,	# Tuesday
1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,	# Wednesday
0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,	# Thursday
1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,	# Friday
0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,	# Saturday
1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1	# Sunday
)
# a vector for the qualifications
qualification=c(1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1)  # qualifications of the workers for an arbitrary working day
qualification.v=c(rep(qualification, times=7))  # 7 times repeated


# second objective function (constraints and weighting)
obj_2 = rep(2, times = (196))
# violating the preferences of the workers is weighted with "-1"
obj_2[which(preference.v==0)]=-1
# violating the qualifications of the workers is weighted with "-2"
obj_2[which(qualification.v==0)]=-2


# the equalities or inequalities of constraints
dir = c(rep("<=",times=7),	    # shifts that could be done by the workers
        rep(">=",times=7),	    # minimum number of shifts, which should be done by workers - in this example 3 shifts are the minimum
        rep("<=",times=196),	# late-shift followed by an early-shift is to prevent
        rep("<=",times=28),	    # only one assignment per shift and worker
        rep("<=",times=98),	    # a signle worker can only work in one early-shift respectively in one late-shift
        rep("==",times=2)	    # long weekend
       )

# right-hand-side-values
rhs = c(rep(14,times=7),	# shifts that could be done by the workers
        rep(3,times=7),		# minimum number of shifts, which should be done by workers - in this example 3 shifts are the minimum
        rep(1,times=196),	# late-shift followed by an early-shift is to prevent
        rep(1,times=28),	# only one assignment per shift and worker
        rep(1,times=98),	# a signle worker can only work in one early-shift respectively in one late-shift
        rep(0,times=2)		# long weekend
       )

# Here maximization
max = TRUE

# here we need binary variables
types = rep("B",times=196)

In [26]:
# execute the optimization
res_1=Rglpk_solve_LP(obj_1, mat, dir, rhs, types = types, max = max)
res_2=Rglpk_solve_LP(obj_2, mat[8:338,], dir[8:338], rhs[8:338], types = types, max = max)

**Part 4**: Looking at the violations (which only occure in `res_1`)

And now controll the results - let's see where are the violations.

In [32]:
# output the weekplan of all the workers
for(l in 0:6) {

	h=eval(parse(text=paste0("	c(((0*7*4)+(",l,"*4+1)):((0*7*4)+(",l,"*4+4)),			# Monday     of worker l 
                                  ((1*7*4)+(",l,"*4+1)):((1*7*4)+(",l,"*4+4)), 			# Tuesday    of worker l 
                                  ((2*7*4)+(",l,"*4+1)):((2*7*4)+(",l,"*4+4)), 			# Wednesday  of worker l 
                                  ((3*7*4)+(",l,"*4+1)):((3*7*4)+(",l,"*4+4)), 			# Thursday   of worker l 
                                  ((4*7*4)+(",l,"*4+1)):((4*7*4)+(",l,"*4+4)), 			# Friday     of worker l 
                                  ((5*7*4)+(",l,"*4+1)):((5*7*4)+(",l,"*4+4)), 			# Saturday   of worker l 
                                  ((6*7*4)+(",l,"*4+1)):((6*7*4)+(",l,"*4+4)))	")  ))	# Sunday     of worker l 


	# printing out the weekplan of the workers
	h.m = matrix(res_1$solution[h],ncol=4,byrow=TRUE)  # weekplan of all workers of res_1 (here are violations)
	rownames(h.m)=c("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
    colnames(h.m)=c("ES_L1","LS_L1","ES_L2","LS_L2")
    print(paste0("worker_",l+1))
    print(h.m)
    print("")

}  # Ende for-Schleife

[1] "worker_1"
          ES_L1 LS_L1 ES_L2 LS_L2
Monday        0     0     0     0
Tuesday       0     0     0     0
Wednesday     0     0     0     0
Thursday      0     1     0     0
Friday        0     1     0     0
Saturday      0     0     1     1
Sunday        1     0     0     0
[1] ""
[1] "worker_2"
          ES_L1 LS_L1 ES_L2 LS_L2
Monday        0     0     1     1
Tuesday       0     0     0     1
Wednesday     0     0     0     0
Thursday      0     0     0     0
Friday        0     0     1     1
Saturday      0     0     0     0
Sunday        0     1     0     0
[1] ""
[1] "worker_3"
          ES_L1 LS_L1 ES_L2 LS_L2
Monday        0     0     0     0
Tuesday       0     1     1     0
Wednesday     0     0     0     0
Thursday      0     0     1     0
Friday        0     0     0     0
Saturday      0     0     0     0
Sunday        0     0     0     1
[1] ""
[1] "worker_4"
          ES_L1 LS_L1 ES_L2 LS_L2
Monday        0     1     0     0
Tuesday       0     0     0     0
W

And here in `res_1` there are some constraint violations - where we will have a look now on it

worker_1:

* 2 violations of late shift to early shift
    * Friday LS_L1 to saturday ES_L2
    * Saturday LS_L2 to sunday ES_L1
* 2 violations in preferences
    * Saturday ES_L2 and LS_L2
    
worker_2:

* 1 violation of late shift to early shift
    * Sunday LS_L1 to monday ES_L2
* violation of qualification
    * Sunday LS_L1
    
worker_3:

* 1 violation in preferences
    * Sunday LS_L2
    
worker_4:

* violation of qualification
    * Sunday ES_L2

And in `res_2` there are no violations - as you can see...

In [31]:
# output the weekplan of all the workers
for(l in 0:6) {

	h=eval(parse(text=paste0("	c(((0*7*4)+(",l,"*4+1)):((0*7*4)+(",l,"*4+4)),			# Monday     of worker l 
                                  ((1*7*4)+(",l,"*4+1)):((1*7*4)+(",l,"*4+4)), 			# Tuesday    of worker l 
                                  ((2*7*4)+(",l,"*4+1)):((2*7*4)+(",l,"*4+4)), 			# Wednesday  of worker l 
                                  ((3*7*4)+(",l,"*4+1)):((3*7*4)+(",l,"*4+4)), 			# Thursday   of worker l 
                                  ((4*7*4)+(",l,"*4+1)):((4*7*4)+(",l,"*4+4)), 			# Friday     of worker l 
                                  ((5*7*4)+(",l,"*4+1)):((5*7*4)+(",l,"*4+4)), 			# Saturday   of worker l 
                                  ((6*7*4)+(",l,"*4+1)):((6*7*4)+(",l,"*4+4)))	")  ))	# Sunday     of worker l 


	# printing out the weekplan of the workers
	h.m = matrix(res_2$solution[h],ncol=4,byrow=TRUE)  # weekplan of all workers of res_2 (here are no violations)
	rownames(h.m)=c("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
    colnames(h.m)=c("ES_L1","LS_L1","ES_L2","LS_L2")
    print(paste0("worker_",l+1))
    print(h.m)
    print("")

}  # Ende for-Schleife

[1] "worker_1"
          ES_L1 LS_L1 ES_L2 LS_L2
Monday        0     0     0     0
Tuesday       0     0     0     0
Wednesday     0     0     0     0
Thursday      0     1     0     0
Friday        0     1     0     0
Saturday      0     0     0     0
Sunday        0     0     1     0
[1] ""
[1] "worker_2"
          ES_L1 LS_L1 ES_L2 LS_L2
Monday        0     0     1     1
Tuesday       0     0     0     1
Wednesday     0     0     0     0
Thursday      0     0     0     0
Friday        0     0     1     1
Saturday      0     0     0     0
Sunday        0     0     0     0
[1] ""
[1] "worker_3"
          ES_L1 LS_L1 ES_L2 LS_L2
Monday        0     0     0     0
Tuesday       0     1     1     0
Wednesday     0     0     0     0
Thursday      0     0     1     0
Friday        0     0     0     0
Saturday      0     0     0     0
Sunday        1     0     0     0
[1] ""
[1] "worker_4"
          ES_L1 LS_L1 ES_L2 LS_L2
Monday        0     1     0     0
Tuesday       0     0     0     0
W